## Case Study 04
###Machine Learning to predict public sentiment from text data.
Look into twitter text data to predict if the given text has positive or negative sentiment
towards a particular brand. The dataset includes twitter text related to Apple and Google
products with user sentiment ranked between ‘positive’, ‘negative’, ‘neutral’ and ‘no_idea’,
sentiments. Create a simpleRNN or LSTM based classifiers to classify tweets into the four
classes. You can avoid ‘emotion_in_tweet_is_directed_at’ column

### Importing of Libraries and dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',encoding= 'unicode_escape')

In [3]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
dt = data.drop(['emotion_in_tweet_is_directed_at'],axis=1)

In [5]:
dt.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


### Preprocessing 

In [6]:
dt.columns = ["Text", "Label"]
dt.head()

,Text,Label
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [7]:
dt['Label'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: Label, dtype: int64

In [8]:
dt['Label']= dt['Label'].map({'No emotion toward brand or product':'neutral','Positive emotion':'positive','Negative emotion':'negative',"I can't tell":'no idea'})

In [9]:
dt['Label'] = dt['Label'].map({'neutral':2,'positive':3,'negative':1,'no idea':0})

In [10]:
dt.head()

,Text,Label
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3
2,@swonderlin Can not wait for #iPad 2 also. The...,3
3,@sxsw I hope this year's festival isn't as cra...,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3


In [11]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9092 non-null   object
 1   Label   9093 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.2+ KB


In [12]:
dt = dt.dropna()

In [13]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9092 non-null   object
 1   Label   9092 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 213.1+ KB


In [14]:
import gensim
dt['text_clean_gensim'] = dt['Text'].apply(lambda x: gensim.utils.simple_preprocess(x))
dt.head()

,Text,Label,text_clean_gensim
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1,"[wesley, have, iphone, after, hrs, tweeting, a..."
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3,"[jessedee, know, about, fludapp, awesome, ipad..."
2,@swonderlin Can not wait for #iPad 2 also. The...,3,"[swonderlin, can, not, wait, for, ipad, also, ..."
3,@sxsw I hope this year's festival isn't as cra...,1,"[sxsw, hope, this, year, festival, isn, as, cr..."
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3,"[sxtxstate, great, stuff, on, fri, sxsw, maris..."


### Fitting RNN model

In [15]:
from keras.preprocessing import text
tokenizer = text.Tokenizer() 
tokenizer.fit_on_texts(list(dt['text_clean_gensim']))
tokenized_texts = tokenizer.texts_to_sequences(dt['text_clean_gensim'])

In [16]:
from keras.utils import pad_sequences
X = pad_sequences(tokenized_texts, maxlen=100)

In [17]:
len(tokenizer.word_index)

9283

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dt['Label'].values, test_size=0.2)

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

In [20]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim = 128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  

In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1188352   
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                1390      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 204       
                                                                 
Total params: 1,190,496
Trainable params: 1,190,496
Non-

In [23]:
model.fit(X_train, y_train, epochs=15, validation_split=0.1)

Epoch 1/15
205/205 [==============================] - 10s 43ms/step - loss: 1.0247 - accuracy: 0.5597 - val_loss: 0.8861 - val_accuracy: 0.5948
Epoch 2/15
205/205 [==============================] - 8s 41ms/step - loss: 0.8885 - accuracy: 0.6044 - val_loss: 0.8438 - val_accuracy: 0.6058
Epoch 3/15
205/205 [==============================] - 9s 43ms/step - loss: 0.7789 - accuracy: 0.6714 - val_loss: 0.8241 - val_accuracy: 0.6374
Epoch 4/15
205/205 [==============================] - 8s 41ms/step - loss: 0.6568 - accuracy: 0.7427 - val_loss: 0.8387 - val_accuracy: 0.6401
Epoch 5/15
205/205 [==============================] - 9s 42ms/step - loss: 0.5701 - accuracy: 0.7869 - val_loss: 0.8769 - val_accuracy: 0.6470
Epoch 6/15
205/205 [==============================] - 9s 42ms/step - loss: 0.5037 - accuracy: 0.8206 - val_loss: 0.9428 - val_accuracy: 0.6168
Epoch 7/15
205/205 [==============================] - 9s 45ms/step - loss: 0.4570 - accuracy: 0.8445 - val_loss: 1.0249 - val_accuracy: 0.636

In [24]:
y_pred = model.predict(X_test)

57/57 [==============================] - 1s 8ms/step


#### We obtained a model witharound 61 % accuracy

### Testing of model

In [37]:
# Given labels : {'neutral':2,'positive':3,'negative':1,'no idea':0}
#In the array order for labels : [No idea, Negative, Neutral, Positive]

In [32]:
test1 = "its very good"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
out = model.predict(test2)
out

1/1 [==============================] - 0s 23ms/step


array([[0.05835051, 0.31404725, 0.14667328, 0.48092896]], dtype=float32)

Here probability is more for 4th value . which means the analysis is "Positive". Here the analysis is correctly predicted

In [33]:
test1 = "its very bad"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
out = model.predict(test2)
out

1/1 [==============================] - 0s 34ms/step


array([[0.07445559, 0.5096694 , 0.10087409, 0.315001  ]], dtype=float32)

Here probability is more for 2nd value . which means the analysis is "Negative". Here the analysis is correctly predicted

In [34]:
test1 = "Sharing the link"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
out = model.predict(test2)
out

1/1 [==============================] - 0s 40ms/step


array([[0.00415804, 0.00301865, 0.88645434, 0.10636897]], dtype=float32)

Here probability is more for 3rd value . which means the analysis is "Neutral". Here the analysis is correctly predicted